In [1]:
import requests
from time import sleep
import pandas as pd
import numpy as np

from keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dense, Activation, BatchNormalization, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
USER_ID = '9G08LOYFU88BJ8GHNRU3'

In [3]:
#http://35.180.16.68/predict?user_id=aaaa&predicted_score=0.761
#http://35.180.16.68/reset?user_id=aaaa

# First Environment

In [ ]:
env = 'http://52.47.62.31/'

In [4]:
r = requests.get(url=env+'reset', params= {'user_id':USER_ID})
data = r.json()
nb_items = data['nb_items']
nb_users = data['nb_users']
next_item = data['next_item']
next_user = data['next_user']
item_history = data['item_history']
user_history = data['user_history']
rating_history = data['rating_history']

In [5]:
ratings = pd.DataFrame(data={'user_id':user_history, 'item_id':item_history, 'rating': rating_history})

In [6]:
ratings.head()

,user_id,item_id,rating
0,29,296,3
1,0,282,2
2,17,270,1
3,62,81,4
4,32,87,5


In [7]:
user_id_input = Input(shape=[1],name='user')
item_id_input = Input(shape=[1], name='item')

embedding_size = 5
user_embedding = Embedding(output_dim=embedding_size, input_dim=nb_users + 1,
                           input_length=1, name='user_embedding')(user_id_input)

item_embedding = Embedding(output_dim=embedding_size, input_dim=nb_items + 1,
                           input_length=1, name='item_embedding')(item_id_input)

user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)

y = Dot(axes=1)([user_vecs, item_vecs])

model = Model(inputs=[user_id_input, item_id_input], outputs=y)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 5)         505         user[0][0]                       
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 5)         1505        item[0][0]                       
__________________________________________________________________________________________________
flatten_1 

In [9]:
model.compile(optimizer='adam', loss='MSE')

In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit([ratings['user_id'], ratings['item_id']], ratings['rating'],
                    batch_size=64, epochs=20, validation_split=0.1,
                    shuffle=True, callbacks=[early_stopping])

Train on 9000 samples, validate on 1000 samples
Epoch 1/20
9000/9000 [==============================] - 0s 28us/step - loss: 9.6570 - val_loss: 9.9388
Epoch 2/20
9000/9000 [==============================] - 0s 14us/step - loss: 9.6160 - val_loss: 9.8043
Epoch 3/20
9000/9000 [==============================] - 0s 13us/step - loss: 9.2005 - val_loss: 9.0123
Epoch 4/20
9000/9000 [==============================] - 0s 14us/step - loss: 8.0223 - val_loss: 7.4776
Epoch 5/20
9000/9000 [==============================] - 0s 14us/step - loss: 6.3231 - val_loss: 5.6894
Epoch 6/20
9000/9000 [==============================] - 0s 14us/step - loss: 4.6332 - val_loss: 4.0981
Epoch 7/20
9000/9000 [==============================] - 0s 14us/step - loss: 3.2487 - val_loss: 2.8645
Epoch 8/20
9000/9000 [==============================] - 0s 14us/step - loss: 2.2407 - val_loss: 2.0039
Epoch 9/20
9000/9000 [==============================] - 0s 14us/step - loss: 1.5628 - val_loss: 1.4360
Epoch 10/20
9000/9000 [==

In [11]:
nb_samples = 1000
mse = 0
users_list = []
ratings_list = []
items_list = []
for i in range(nb_samples):
    sleep(0.05)
    predicted_score = model.predict([[next_user], [next_item]])[0,0]
    r = requests.get(url=env + 'predict', params= {'user_id':USER_ID, 'predicted_score':predicted_score})
    true_rating = r.json()['rating']
    mse += (true_rating - predicted_score)**2
    users_list += [next_user]
    ratings_list += [true_rating]
    items_list += [next_item]
    if (i+1)%100 == 0:
        model.fit([users_list,items_list], ratings_list, epochs=20)
        users_list = []
        ratings_list = []
        items_list = []
    next_item = r.json()['next_item']
    next_user = r.json()['next_user']
print('mse: ', mse/nb_samples )

Train on 90 samples, validate on 10 samples
Epoch 1/20
90/90 [==============================] - 0s 57us/step - loss: 0.5115 - val_loss: 0.2508
Epoch 2/20
90/90 [==============================] - 0s 95us/step - loss: 0.5013 - val_loss: 0.2497
Epoch 3/20
90/90 [==============================] - 0s 87us/step - loss: 0.4852 - val_loss: 0.2486
Epoch 4/20
90/90 [==============================] - 0s 77us/step - loss: 0.4652 - val_loss: 0.2480
Epoch 5/20
90/90 [==============================] - 0s 76us/step - loss: 0.4452 - val_loss: 0.2473
Epoch 6/20
90/90 [==============================] - 0s 122us/step - loss: 0.4200 - val_loss: 0.2466
Epoch 7/20
90/90 [==============================] - 0s 82us/step - loss: 0.3988 - val_loss: 0.2460
Epoch 8/20
90/90 [==============================] - 0s 80us/step - loss: 0.3768 - val_loss: 0.2454
Epoch 9/20
90/90 [==============================] - 0s 137us/step - loss: 0.3572 - val_loss: 0.2449
Epoch 10/20
90/90 [==============================] - 0s 106us/s

# Second Environment

In [3]:
env = 'http://35.180.254.42/'

In [4]:
r = requests.get(url=env+'reset', params= {'user_id':USER_ID})
sleep(0.05)
data = r.json()
nb_items = data['nb_items']
nb_users = data['nb_users']
next_item = data['next_item']
next_user = data['next_user']
next_variables = data['next_variables']
item_history = data['item_history']
user_history = data['user_history']
rating_history = data['rating_history']
variables_history = np.array(data['variables_history'])

In [5]:
ratings = pd.DataFrame(data={'user_id':user_history, 'item_id':item_history, 'rating': rating_history, 'variable0': variables_history[:,0], 'variable1': variables_history[:,1], 'variable2': variables_history[:,2], 'variable3': variables_history[:,3], 'variable4': variables_history[:,4]})

In [6]:
ratings.head()

,user_id,item_id,rating,variable0,variable1,variable2,variable3,variable4
0,61,48,5,0.539598,3.106783,1.523826,-0.049320,0.388647
1,27,64,5,1.795906,1.402216,-0.018452,1.368225,0.612891
2,9,193,2,0.997335,-0.462516,1.423192,0.199319,0.529538
3,41,106,2,1.068070,1.349626,2.210689,0.720025,0.842338
4,48,86,5,0.645593,0.339054,2.191272,2.452960,0.573745


In [42]:
user_id_input = Input(shape=[1],name='user')
item_id_input = Input(shape=[1], name='item')
variables_input = Input(shape=[5], name='variables')

embedding_size = 5
user_embedding = Embedding(output_dim=embedding_size, input_dim=nb_users + 1,
                           input_length=1, name='user_embedding')(user_id_input)

item_embedding = Embedding(output_dim=embedding_size, input_dim=nb_items + 1,
                           input_length=1, name='item_embedding')(item_id_input)

user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)
#variables_vecs = Flatten()(variables_input)

x = Concatenate()([user_vecs, item_vecs, variables_input])

x = Dense(100)(x)
x = Activation('relu')(x)

x = Dropout(0.4)(x)

x = Dense(80)(x)
x = Activation('relu')(x)

x = Dropout(0.4)(x)

x = Dense(60)(x)
x = Activation('relu')(x)

x = Dense(1)(x)


model = Model(inputs=[user_id_input, item_id_input, variables_input], outputs=x)

In [43]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 5)         505         user[0][0]                       
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 5)         1505        item[0][0]                       
__________________________________________________________________________________________________
flatten_17

In [44]:
model.compile(optimizer='adam', loss='MSE')

In [45]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit([ratings['user_id'], ratings['item_id'], ratings[['variable0', 'variable1', 'variable2', 'variable3', 'variable4']]], ratings['rating'],
                    batch_size=64, epochs=20, validation_split=0.1,
                    shuffle=True, callbacks=[early_stopping])

Train on 1800 samples, validate on 200 samples
Epoch 1/20
1800/1800 [==============================] - 1s 468us/step - loss: 4.6661 - val_loss: 2.4030
Epoch 2/20
1800/1800 [==============================] - 0s 40us/step - loss: 2.6224 - val_loss: 2.1989
Epoch 3/20
1800/1800 [==============================] - 0s 43us/step - loss: 2.4124 - val_loss: 2.1272
Epoch 4/20
1800/1800 [==============================] - 0s 42us/step - loss: 2.2081 - val_loss: 1.9871
Epoch 5/20
1800/1800 [==============================] - 0s 39us/step - loss: 2.1261 - val_loss: 1.9306
Epoch 6/20
1800/1800 [==============================] - 0s 44us/step - loss: 1.8640 - val_loss: 1.7185
Epoch 7/20
1800/1800 [==============================] - 0s 42us/step - loss: 1.6123 - val_loss: 1.4989
Epoch 8/20
1800/1800 [==============================] - 0s 43us/step - loss: 1.2880 - val_loss: 1.2539
Epoch 9/20
1800/1800 [==============================] - 0s 43us/step - loss: 1.0764 - val_loss: 1.1049
Epoch 10/20
1800/1800 [==

In [21]:
nb_samples = 1000
mse = 0
users_list = []
ratings_list = []
items_list = []
variables_list = []
for i in range(nb_samples):
    sleep(0.05)
    predicted_score = model.predict([[next_user], [next_item], [next_variables]])[0,0]
    r = requests.get(url=env + 'predict', params= {'user_id':USER_ID, 'predicted_score':predicted_score})
    true_rating = r.json()['rating']
    mse += (true_rating - predicted_score)**2
    users_list += [next_user]
    ratings_list += [true_rating]
    items_list += [next_item]
    variables_list += [next_variables]
    if (i+1)%100 == 0:
        model.fit([users_list, items_list, variables_list], ratings_list, epochs=20)
        users_list = []
        ratings_list = []
        items_list = []
        variables_list = []
    next_item = r.json()['next_item']
    next_user = r.json()['next_user']
    next_variables = r.json()['next_variables']
print('mse: ', mse/nb_samples )

Epoch 1/20
100/100 [==============================] - 0s 395us/step - loss: 1.4108
Epoch 2/20
100/100 [==============================] - 0s 428us/step - loss: 1.3346
Epoch 3/20
100/100 [==============================] - 0s 486us/step - loss: 1.0964
Epoch 4/20
100/100 [==============================] - 0s 515us/step - loss: 1.1536
Epoch 5/20
100/100 [==============================] - 0s 477us/step - loss: 0.8986
Epoch 6/20
100/100 [==============================] - 0s 498us/step - loss: 0.7134
Epoch 7/20
100/100 [==============================] - 0s 433us/step - loss: 0.6050
Epoch 8/20
100/100 [==============================] - 0s 470us/step - loss: 0.6539
Epoch 9/20
100/100 [==============================] - 0s 518us/step - loss: 0.7286
Epoch 10/20
100/100 [==============================] - 0s 446us/step - loss: 0.6381
Epoch 11/20
100/100 [==============================] - 0s 443us/step - loss: 0.6801
Epoch 12/20
100/100 [==============================] - 0s 562us/step - loss: 0.5421
E

100/100 [==============================] - 0s 428us/step - loss: 0.4451
Epoch 20/20
100/100 [==============================] - 0s 483us/step - loss: 0.5255
Epoch 1/20
100/100 [==============================] - 0s 389us/step - loss: 1.0631
Epoch 2/20
100/100 [==============================] - 0s 354us/step - loss: 1.2484
Epoch 3/20
100/100 [==============================] - 0s 420us/step - loss: 0.9313
Epoch 4/20
100/100 [==============================] - 0s 394us/step - loss: 0.8696
Epoch 5/20
100/100 [==============================] - 0s 440us/step - loss: 0.7149
Epoch 6/20
100/100 [==============================] - 0s 378us/step - loss: 0.6416
Epoch 7/20
100/100 [==============================] - 0s 446us/step - loss: 0.7130
Epoch 8/20
100/100 [==============================] - 0s 450us/step - loss: 0.8459
Epoch 9/20
100/100 [==============================] - 0s 364us/step - loss: 0.7044
Epoch 10/20
100/100 [==============================] - 0s 386us/step - loss: 0.6014
Epoch 11/20
1

100/100 [==============================] - 0s 414us/step - loss: 0.5655
Epoch 18/20
100/100 [==============================] - 0s 442us/step - loss: 0.5001
Epoch 19/20
100/100 [==============================] - 0s 394us/step - loss: 0.4804
Epoch 20/20
100/100 [==============================] - 0s 373us/step - loss: 0.4103
mse:  0.7444024174672361
